In [1]:
from collections import namedtuple
# neptune
import os
import numpy as np

from time import sleep
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.nn import functional as F
from transformers import  T5ForConditionalGeneration, T5Config, get_linear_schedule_with_warmup, AdamW
from project.data.data import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gersa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
t = torch.rand((3,2))
t

tensor([[0.8504, 0.9204],
        [0.2023, 0.7988],
        [0.6397, 0.8530]])

In [3]:
torch.argmax(t.transpose(0,1), dim=0)

tensor([1, 1, 1])

# Data processing

In [4]:
run_preprocessing = False
# No need to rerun the processing

if run_preprocessing:
    processor = RaceDataProcessor()
    processor.process_data("RACE/", "Processed_New/")
else:
    print("Data Processing: skip")

Data Processing: skip


In [5]:
class Args:
    def __init__(self):
        self.data_path =  "D:\Github/decepticon/Processed_New"
        self.batch_size =  16
        self.num_workers =  0
        self.pretrained_model = "t5-small"
        self.tokenizer_len = 0
        self.padding_id = 0
    def to_dict(self):
        out = dict()
        out["data_path"] = self.data_path
        out["batch_size"] = self.batch_size
        out["num_workers"] = self.num_workers
        out["pretrained_model"] = self.pretrained_model
        out["tokenizer_len"] = self.tokenizer_len
        out["weight_decay"] = self.weight_decay
        out["learning_rate"] = self.learning_rate
        out["padding_id"] = self.padding_id
        return out

hparams=Args()

In [7]:
data_module = RaceDataModule(hparams)
## Colab is weird, so  I have to put it here
#data_module.tokenizer.add_special_tokens({'additional_special_tokens': ['<answer>', '<context>']})
print(len(data_module.tokenizer))
hparams.tokenizer_len = len(data_module.tokenizer)

data_module.prepare_data()
data_module.setup()

32102
SETUP: Training Dataset
SETUP: Validation Dataset
SETUP: Test Dataset


32102

In [8]:
model = T5FinetuneForRACE(hparams.to_dict())
trainer = Trainer(accumulate_grad_batches=4)
trainer.fit(model, data_module)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
C:\Users\gersa\.conda\envs\network\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
241.969   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

COLLATE based of t5-small
COLLATE based of t5-small
COLLATE based of t5-small


Training: 0it [00:00, ?it/s]

COLLATE based of t5-small
COLLATE based of t5-small
COLLATE based of t5-small
COLLATE based of t5-small
COLLATE based of t5-small


C:\Users\gersa\.conda\envs\network\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [7]:
class T5FinetuneForRACE(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FinetuneForRACE, self).__init__()
        self.save_hyperparameters()
        self.hparams = hparams
        if self.hparams.pretrained_model in ["t5-base","t5-small":
            config = T5Config(decoder_start_token_id=0)
            self.model = T5ForConditionalGeneration(config).from_pretrained(self.hparams.pretrained_model)
            self.model.resize_token_embeddings(self.hparams.tokenizer_len)
        else:
            raise NotImplementedError
    def mask_label_padding(self, labels):
        MASK_ID = -100
        labels[labels == self.hparams.padding_id] = MASK_ID 
        return labels
 
    def forward(self, ids, mask, labels):
        return self.model(input_ids = ids, attention_mask = mask, labels = labels)


    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self(x["input_ids"], x["attention_mask"], self.mask_label_padding(y["input_ids"]))
        loss = output.loss
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x["input_ids"], x["attention_mask"], self.mask_label_padding(y["input_ids"]))
        loss = output[0]
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        output = self(x["input_ids"], x["attention_mask"], self.mask_label_padding(y["input_ids"]))
        loss = output[0]
        return loss

    def configure_optimizers(self):
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [{'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)],
                                         'weight_decay': 0.01}, 
                                        {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)],
                                         'weight_decay': 0.0}]
        
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr = self.hparams.learning_rate)
#         scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=0,
#             # Default value in run_glue.py
#             num_training_steps=self.hparams.num_training_steps)
 
        return [optimizer]#, [scheduler]

In [7]:
len([32100,   432,     8,   756,     5, 32101,   932,   305,    19,  4318,
          4351,    31,     7,  1430,     5,    86,   685,     6,    34,    19,
          1086,   718,  7508,    31,     7,  1430,   250,    34,    19,     3,
          4894,  9443,    41,     3,    61,    57,  5234,     5,    37,  5216,
            13,  7508,    31,     7,  1430,    65,     3,     9,   307,   892,
             5,    94,    19,   243,    24,     8,  3994,   639,    45,     8,
         10282, 17733,  3397,    16,  1473,     5,   461,  7508,    31,     7,
          1430,     6,  4318,  5234,     3,  1544,     3,     9,   534,   773,
            13,  6605,  4340,     5,    94,    19,  2303,    28,     3,     9,
          8384,    11,  3353,    28, 15431, 11388,    41,     3,   137,  4351,
           333,    12,     3,  1544,    34,   182,   231,     5,   461,    48,
           239,     6,   502,   278,    31,    17,    43,    12,   281,    12,
           496,     5,  1698,   384,    28,     3,     9,  4940,  5168,     7,
            95,  1450,   443,   102,    18,  6489,  5692,    41,     3,   137,
            37,  5692,     7,    33,  1086,   386,  2602,    10,  1001,     6,
          1131,    11,  1692,     5,  1589,  5024,    21,     8,  2353,     6,
          1131,  5024,    21,     8,  2039,    11,  1692,  5024,    21,     8,
           520,     5,  4318,  1362,   857,    24,     8,  5692,     7,    56,
           830,    70,  5234,   207,  5851,    11,   428,   135, 11578,    11,
           579,     5,     1])

193

In [14]:
[data_module.tokenizer.decode(t) for t in i["articles"]["input_ids"]]

["<answer> All the above.<context> May 5 is Japanese Children's Day. In fact, it is usually called Boy's Day because it is mainly celebrated ( ) by boys. The celebration of Boy's Day has a long history. It is said that the festival comes from the Dragon Boat Festival in China. On Boy's Day, Japanese boys eat a special kind of rice cake. It is covered with a leaf and filled with bean paste ( ). Children love to eat it very much. On this day, children don't have to go to school. Each family with a boy hangs up huge carp-shaped flag ( ). The flags are usually three colors: black, red and blue. Black stands for the father, red stands for the mother and blue stands for the son. Japanese parents believe that the flags will bring their boys good luck and give them courage and power.</s>"]

In [28]:
for x, y in data_module.train_dataloader():
    break

In [29]:
x["attention_mask"]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [32]:
next(data_module.train_dataloader())

TypeError: 'DataLoader' object is not an iterator

In [16]:
data_module.train_dataloader().collate_fn(i)

COLLATE based of t5-base


TypeError: string indices must be integers